In [1]:
using ITensors
using Statistics
using QuantumNaturalGradient

In [2]:
using QuantumNaturalfPEPS

[ Info: Precompiling QuantumNaturalfPEPS [8cccb88f-824b-4ada-bd77-4bd123faaa4e]


In [3]:
# PEPS(Lx,Ly,phys_dim,bond_dim) generates a PEPS with dimensions Lx * Ly, physical dimension = phys_dim and bond dimension bond_dim
peps = PEPS(3,3,2,1)

PEPS(ITensor[ITensor ord=3
Dim 1: (dim=1|id=388|"ind_1")
Dim 2: (dim=1|id=549|"ind_7")
Dim 3: (dim=2|id=84|"phys_1_1")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1×2
[:, :, 1] =
 -0.05395004141629708

[:, :, 2] =
 -0.7598629013849278 ITensor ord=4
Dim 1: (dim=1|id=549|"ind_7")
Dim 2: (dim=1|id=415|"ind_10")
Dim 3: (dim=1|id=656|"ind_3")
Dim 4: (dim=2|id=13|"phys_2_1")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1×1×2
[:, :, 1, 1] =
 0.8529848023130183

[:, :, 1, 2] =
 1.0290627425810266 ITensor ord=3
Dim 1: (dim=1|id=415|"ind_10")
Dim 2: (dim=1|id=893|"ind_5")
Dim 3: (dim=2|id=28|"phys_3_1")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1×2
[:, :, 1] =
 -0.10293910917015826

[:, :, 2] =
 -0.42662781710523756; ITensor ord=4
Dim 1: (dim=1|id=388|"ind_1")
Dim 2: (dim=1|id=75|"ind_2")
Dim 3: (dim=1|id=826|"ind_8")
Dim 4: (dim=2|id=72|"phys_1_2")
NDTensors.Dense{Float64, Vector{Float64}}
 1×1×1×2
[:, :, 1, 1] =
 0.31712234027590663

[:, :, 1, 2] =
 1.5782155292175333 ITensor ord=5
Dim 1: 

In [4]:
# You can change the entries of the PEPS by using write!
write!(peps, complex(rand(length(peps))))

In [18]:
# Construct a Hamiltonian using OpSum()
# example: Heisenberg

Nx = size(peps)[1]
Ny = size(peps)[2]

hilbert = reshape(siteinds("S=1/2",Nx* Ny), Nx,Ny)

ham_heisenberg = OpSum()
for i in 1:Nx-1
    for j in 1:Ny-1
        ham_heisenberg += (-1,"X",(i,j),"X",(i,j+1)) #structure: (prefactor, operator, position which it acts on, operator, position)
        ham_heisenberg += (-1,"X",(i,j),"X",(i+1,j))
        ham_heisenberg += (-1,"Y",(i,j),"Y",(i,j+1))
        ham_heisenberg += (-1,"Y",(i,j),"Y",(i+1,j))
        ham_heisenberg += (-1,"Z",(i,j),"Z",(i,j+1))
        ham_heisenberg += (-1,"Z",(i,j),"Z",(i+1,j))
    end
end

ham_heisenberg_op = QuantumNaturalGradient.TensorOperatorSum(ham_heisenberg, hilbert);

In [11]:
# Construct a Hamiltonian using OpSum()
# easy example: only -Sz so the ground state should have energy -9 

Nx = size(peps)[1]
Ny = size(peps)[2]

hilbert = reshape(siteinds("S=1/2",Nx* Ny), Nx,Ny)

ham = OpSum()
for i in 1:Nx
    for j in 1:Ny
        ham += (-1,"Z",(i,j))
    end
end

ham_op = QuantumNaturalGradient.TensorOperatorSum(ham, hilbert);

In [12]:
# this function generates the function needed for the optimization
ok_ek = generate_Oks_and_Eks(peps, ham) 

(::QuantumNaturalfPEPS.var"#Oks_and_Eks#26"{Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, PEPS, Sum{Scaled{ComplexF64, Prod{Op}}}}) (generic function with 1 method)

In [7]:
# callback function for evolution
function test(;energy_value, model, misc, niter)
    
end

test (generic function with 1 method)

In [16]:
dt = 0.025  # Time step
eigen_cut = 0.1  # Eigenvalue cutoff for solver
integrator = QuantumNaturalGradient.Euler(lr=dt)  # Define the integrator with learning rate
solver = QuantumNaturalGradient.EigenSolver(eigen_cut, verbose=true)  # Eigenvalue solver with verbosity

θ = complex(rand(length(peps))+im.*rand(length(peps)))  # Initialize your parameter vector

# Evolve the system
@time loss_value, trained_θ, misc = QuantumNaturalGradient.evolve(ok_ek, θ; 
                                        integrator, 
                                        verbosity=2, 
                                        solver, 
                                        sample_nr = 100,  # Number of samples 
                                        maxiter = 100,  # Maximum iterations 
                                        callback = test,  # Callback function for each iteration
                                        )

[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.5031888648472831 - max_val: 1.6838348709899382
[ Info: iter 1: EnergySummary(E = 1.12 ± 0.32, var(E) = 10.2 ± 1.4, Nₛ=100), ‖∇f‖ = 3.116541885065591, ‖θ‖ = 3.4285661136757066, tdvp_error = 0.0005562860392309688
[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.4790586941420991 - max_val: 1.4539012650059717
[ Info: iter 2: EnergySummary(E = 1.16 ± 0.26, var(E) = 6.76 ± 0.98, Nₛ=100), ‖∇f‖ = 3.122217491720126, ‖θ‖ = 3.4294512850704257, tdvp_error = 0.000778805671809657
[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.5384647553987669 - max_val: 2.0342597724894493
[ Info: iter 3: EnergySummary(E = 0.26 ± 0.27, var(E) = 7.16 ± 0.99, Nₛ=100), ‖∇f‖ = 3.115833303659241, ‖θ‖ = 3.4303394537512166, tdvp_error = 0.00400670852897167
[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.5137989229057052 - max_val: 1.6863857031636391
[ Info: iter 4: EnergySummary(E = 0.02 ± 0.29, var(E) = 8.7 ± 1.3, Nₛ=100), ‖∇f‖ = 3.12134431

[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.6376377049785154 - max_val: 1.748029515478885
[ Info: iter 25: EnergySummary(E = -6.3 ± 0.19, var(E) = 3.51 ± 0.45, Nₛ=100), ‖∇f‖ = 2.1189962722165645, ‖θ‖ = 3.446144929989426, tdvp_error = 0.01662470858407028
[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.5740910598679839 - max_val: 1.8016244788803648
[ Info: iter 26: EnergySummary(E = -6.96 ± 0.19, var(E) = 3.47 ± 0.49, Nₛ=100), ‖∇f‖ = 2.0935566768916356, ‖θ‖ = 3.4465520769666127, tdvp_error = 0.0036452965126820835
[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.5686299494362963 - max_val: 2.0048679933274984
[ Info: iter 27: EnergySummary(E = -6.8 ± 0.21, var(E) = 4.57 ± 0.8, Nₛ=100), ‖∇f‖ = 2.018333880571183, ‖θ‖ = 3.446949460242704, tdvp_error = 0.0032992486086742545
[ Info: EigenSolver: Null space size: 9 - 50.0%  - cn: 0.5723879255461377 - max_val: 1.7914699643907535
[ Info: iter 28: EnergySummary(E = -7.02 ± 0.18, var(E) = 3.35 ± 0.51, Nₛ=100), ‖∇f‖ = 1.

[ Info: EigenSolver: Null space size: 11 - 61.1%  - cn: 0.5883395326279897 - max_val: 1.7825528061176876
[ Info: iter 49: EnergySummary(E = -8.74 ± 0.068, var(E) = 0.46 ± 0.1, Nₛ=100), ‖∇f‖ = 0.8077899650382709, ‖θ‖ = 3.4506839959787503, tdvp_error = 0.003140666936817471
[ Info: EigenSolver: Null space size: 11 - 61.1%  - cn: 0.4806527362533414 - max_val: 1.9603391710087836
[ Info: iter 50: EnergySummary(E = -8.76 ± 0.065, var(E) = 0.427 ± 0.099, Nₛ=100), ‖∇f‖ = 0.6431365757279311, ‖θ‖ = 3.4507430892484785, tdvp_error = 0.001369527206026011
[ Info: EigenSolver: Null space size: 12 - 66.7%  - cn: 0.6298785137892433 - max_val: 2.948758167025944
[ Info: iter 51: EnergySummary(E = -8.72 ± 0.075, var(E) = 0.57 ± 0.16, Nₛ=100), ‖∇f‖ = 0.7504664499472243, ‖θ‖ = 3.4507805469785433, tdvp_error = 0.015896615378667045
[ Info: EigenSolver: Null space size: 12 - 66.7%  - cn: 0.5180819597011168 - max_val: 1.5800178076246874
[ Info: iter 52: EnergySummary(E = -8.78 ± 0.069, var(E) = 0.48 ± 0.17, Nₛ=1

[ Info: EigenSolver: Null space size: 17 - 94.4%  - cn: NaN - max_val: 2.997875494506249
[ Info: iter 73: EnergySummary(E = -8.98 ± 0.02, var(E) = 0.04 ± 0.039, Nₛ=100), ‖∇f‖ = 0.1149318408015337, ‖θ‖ = 3.4511780605430227, tdvp_error = 1.999997010226906e-6
[ Info: EigenSolver: Null space size: 15 - 83.3%  - cn: 0.4766239480094735 - max_val: 3.684631556030668
[ Info: iter 74: EnergySummary(E = -8.94 ± 0.034, var(E) = 0.118 ± 0.064, Nₛ=100), ‖∇f‖ = 0.2317955054290962, ‖θ‖ = 3.451179256631495, tdvp_error = 0.00021029882421164103
[ Info: EigenSolver: Null space size: 16 - 88.9%  - cn: 0.2468461243691308 - max_val: 2.249996294488004
[ Info: iter 75: EnergySummary(E = -8.96 ± 0.028, var(E) = 0.079 ± 0.054, Nₛ=100), ‖∇f‖ = 0.2062722001834131, ‖θ‖ = 3.4511841217365133, tdvp_error = 9.538069750458789e-6
[ Info: EigenSolver: Null space size: 15 - 83.3%  - cn: 0.3839334399022221 - max_val: 2.853294371635601
[ Info: iter 76: EnergySummary(E = -8.92 ± 0.039, var(E) = 0.155 ± 0.072, Nₛ=100), ‖∇f‖ = 

[ Info: EigenSolver: Null space size: 16 - 88.9%  - cn: 0.8752832679956698 - max_val: 5.522984122193095
[ Info: iter 97: EnergySummary(E = -8.96 ± 0.028, var(E) = 0.079 ± 0.054, Nₛ=100), ‖∇f‖ = 0.17834391614434855, ‖θ‖ = 3.451267554131115, tdvp_error = 0.0017083229828932511
[ Info: EigenSolver: Null space size: 16 - 88.9%  - cn: 0.27188990875588687 - max_val: 4.149042051658863
[ Info: iter 98: EnergySummary(E = -8.96 ± 0.028, var(E) = 0.079 ± 0.054, Nₛ=100), ‖∇f‖ = 0.15349702204282847, ‖θ‖ = 3.451270434100092, tdvp_error = 1.1525766971498541e-5
[ Info: EigenSolver: Null space size: 15 - 83.3%  - cn: 0.336862743833937 - max_val: 5.057167294800621
[ Info: iter 99: EnergySummary(E = -8.94 ± 0.034, var(E) = 0.118 ± 0.064, Nₛ=100), ‖∇f‖ = 0.18832050354289093, ‖θ‖ = 3.451272567492793, tdvp_error = 4.949964229150616e-5
[ Info: EigenSolver: Null space size: 15 - 83.3%  - cn: 0.7468536190659425 - max_val: 6.7947547341292065
[ Info: iter 100: EnergySummary(E = -8.94 ± 0.034, var(E) = 0.118 ± 0.0

 ──────────────────────────────────────────────────────────────────────────────
                                      Time                    Allocations      
                             ───────────────────────   ────────────────────────
      Tot / % measured:           97.8s / 100.0%           52.2GiB / 100.0%    

 Section             ncalls     time    %tot     avg     alloc    %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 integrator             100    97.8s  100.0%   978ms   52.2GiB  100.0%   534MiB
   NaturalGradient      100    97.8s  100.0%   978ms   52.2GiB  100.0%   534MiB
     Oks_and_Eks        100    97.7s   99.9%   977ms   52.2GiB  100.0%   534MiB
     solver             100   54.5ms    0.1%   545μs   7.15MiB    0.0%  73.3KiB
     copy Oks           100   41.3ms    0.0%   413μs   8.38MiB    0.0%  85.8KiB
 ────────────────────────────────────────────────────────────────────────────── 97.825986 seconds (216.85 M allocations

[ Info: evolve: Done


(-8.94, ComplexF64[0.7291279081130918 + 1.017366211196627im, 0.050673686747292 + 0.013901090865963731im, 0.8248789815436334 + 1.0705777901463038im, 0.03746145555416469 + 0.044542287490837544im, 1.0243337324543196 + 0.1993924695517553im, 0.014065551168353776 + 0.055320398179086544im, 0.4645218573855911 + 0.6342479893432859im, 0.03607922570551875 + 0.004234855588938363im, 1.3687592852974995 + 0.44972725574362865im, 0.042465456493800346 + 0.05701689010526223im, 0.7950538686042011 + 0.7543191756607357im, 0.04112598144356766 + 0.021617415187447087im, 0.061121212508836446 + 0.8355617620506136im, 0.015355162675048228 + 0.0350541085016978im, 0.9796953629343788 + 0.7065831277468876im, 0.0309514985764676 + 0.043636337363834855im, 1.0465618056934984 + 0.5038652521017708im, 0.02523761466522068 + 0.033662090237316006im], Dict{String, Any}("niter" => 100, "history" => [1.1200000000000003 10.167272727272724 … 10.167272727272724 0.0005562860392309688; 1.16 6.762020202020204 … 6.762020202020204 0.00077

In [22]:
# As one can see the values corresponding to Spin up are very big
# compared to the ones corresponding to Spin down after optimization
abs.(trained_θ[1:2:end])./abs.(trained_θ[2:2:end])

9-element Vector{Float64}:
 23.82042142705733
 23.221246082572538
 18.28223854397602
 21.641317655912864
 20.265636551609454
 23.588429268767126
 21.89183149851564
 22.578374209981693
 27.608192643030133